In [1]:
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import json
import numpy as np


In [2]:
#!pip install astropy


In [3]:
from astropy.table import Table
import requests
import time
from io import StringIO

In [4]:
ps1filename = "https://ps1images.stsci.edu/cgi-bin/ps1filenames.py"
fitscut = "https://ps1images.stsci.edu/cgi-bin/fitscut.cgi"
 
def getimages(tra, tdec, size=240, filters="grizy", format="fits", imagetypes="stack"):
     
    """Query ps1filenames.py service for multiple positions to get a list of images
    This adds a url column to the table to retrieve the cutout.
     
    tra, tdec = list of positions in degrees
    size = image size in pixels (0.25 arcsec/pixel)
    filters = string with filters to include
    format = data format (options are "fits", "jpg", or "png")
    imagetypes = list of any of the acceptable image types.  Default is stack;
        other common choices include warp (single-epoch images), stack.wt (weight image),
        stack.mask, stack.exp (exposure time), stack.num (number of exposures),
        warp.wt, and warp.mask.  This parameter can be a list of strings or a
        comma-separated string.
 
    Returns an astropy table with the results
    """
     
    if format not in ("jpg","png","fits"):
        raise ValueError("format must be one of jpg, png, fits")
    # if imagetypes is a list, convert to a comma-separated string
    if not isinstance(imagetypes,str):
        imagetypes = ",".join(imagetypes)
    # put the positions in an in-memory file object
    cbuf = StringIO()
    cbuf.write('\n'.join(["{} {}".format(ra, dec) for (ra, dec) in zip(tra,tdec)]))
    cbuf.seek(0)
    # use requests.post to pass in positions as a file
    r = requests.post(ps1filename, data=dict(filters=filters, type=imagetypes),
        files=dict(file=cbuf))
    r.raise_for_status()
    tab = Table.read(r.text, format="ascii")
 
    urlbase = "{}?size={}&format={}".format(fitscut,size,format)
    tab["url"] = ["{}&ra={}&dec={}&red={}".format(urlbase,ra,dec,filename)
            for (filename,ra,dec) in zip(tab["filename"],tab["ra"],tab["dec"])]
    return tab
 
 
if __name__ == "__main__":
    t0 = time.time()
 
    # create a test set of image positions
    tdec = np.append(np.arange(31)*3.95 - 29.1, 88.0)
    tra = np.append(np.arange(31)*12., 0.0)
 
    # get the PS1 info for those positions
    table = getimages(tra,tdec,filters="ri")
    print("{:.1f} s: got list of {} images for {} positions".format(time.time()-t0,len(table),len(tra)))
 
    # extract cutout for each position/filter combination
    for row in table:
        ra = row['ra']
        dec = row['dec']
        projcell = row['projcell']
        subcell = row['subcell']
        filter = row['filter']
 
        # create a name for the image -- could also include the projection cell or other info
        fname = "t{:08.4f}{:+07.4f}.{}.fits".format(ra,dec,filter)
 
        url = row["url"]
        print("%11.6f %10.6f skycell.%4.4d.%3.3d %s" % (ra, dec, projcell, subcell, fname))
        r = requests.get(url)
        open(fname,"wb").write(r.content)
    print("{:.1f} s: retrieved {} FITS files for {} positions".format(time.time()-t0,len(table),len(tra)))

1.6 s: got list of 64 images for 32 positions
   0.000000 -29.100000 skycell.0635.074 t000.0000-29.1000.i.fits
   0.000000 -29.100000 skycell.0635.074 t000.0000-29.1000.r.fits
  12.000000 -25.150000 skycell.0717.077 t012.0000-25.1500.i.fits
  12.000000 -25.150000 skycell.0717.077 t012.0000-25.1500.r.fits
  24.000000 -21.200000 skycell.0802.068 t024.0000-21.2000.i.fits
  24.000000 -21.200000 skycell.0802.068 t024.0000-21.2000.r.fits
  36.000000 -17.250000 skycell.0889.068 t036.0000-17.2500.i.fits
  36.000000 -17.250000 skycell.0889.068 t036.0000-17.2500.r.fits
  48.000000 -13.300000 skycell.0978.067 t048.0000-13.3000.i.fits
  48.000000 -13.300000 skycell.0978.067 t048.0000-13.3000.r.fits
  60.000000  -9.350000 skycell.1069.066 t060.0000-9.3500.i.fits
  60.000000  -9.350000 skycell.1069.066 t060.0000-9.3500.r.fits
  72.000000  -5.400000 skycell.1161.066 t072.0000-5.4000.i.fits
  72.000000  -5.400000 skycell.1161.066 t072.0000-5.4000.r.fits
  84.000000  -1.450000 skycell.1253.064 t084.000

In [5]:
%matplotlib inline
from astropy.io import ascii
from astropy.table import Table

import sys
import re
import numpy as np
import pylab
import json
import requests

try: # Python 3.x
    from urllib.parse import quote as urlencode
    from urllib.request import urlretrieve
except ImportError:  # Python 2.x
    from urllib import pathname2url as urlencode
    from urllib import urlretrieve

try: # Python 3.x
    import http.client as httplib 
except ImportError:  # Python 2.x
    import httplib  

In [7]:
def ps1cone(ra,dec,radius,table="mean",release="dr1",format="csv",columns=None,
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,
           **kw):
    """Do a cone search of the PS1 catalog
    
    Parameters
    ----------
    ra (float): (degrees) J2000 Right Ascension
    dec (float): (degrees) J2000 Declination
    radius (float): (degrees) Search radius (<= 0.5 degrees)
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2)
    """
    
    data = kw.copy()
    data['ra'] = ra
    data['dec'] = dec
    data['radius'] = radius
    return ps1search(table=table,release=release,format=format,columns=columns,
                    baseurl=baseurl, verbose=verbose, **data)


def ps1search(table="mean",release="dr1",format="csv",columns=None,
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs", verbose=False,
           **kw):
    """Do a general search of the PS1 catalog (possibly without ra/dec/radius)
    
    Parameters
    ----------
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    format: csv, votable, json
    columns: list of column names to include (None means use defaults)
    baseurl: base URL for the request
    verbose: print info about request
    **kw: other parameters (e.g., 'nDetections.min':2).  Note this is required!
    """
    
    data = kw.copy()
    if not data:
        raise ValueError("You must specify some parameters for search")
    checklegal(table,release)
    if format not in ("csv","votable","json"):
        raise ValueError("Bad value for format")
    url = "{baseurl}/{release}/{table}.{format}".format(**locals())
    if columns:
        # check that column values are legal
        # create a dictionary to speed this up
        dcols = {}
        for col in ps1metadata(table,release)['name']:
            dcols[col.lower()] = 1
        badcols = []
        for col in columns:
            if col.lower().strip() not in dcols:
                badcols.append(col)
        if badcols:
            raise ValueError('Some columns not found in table: {}'.format(', '.join(badcols)))
        # two different ways to specify a list of column values in the API
        # data['columns'] = columns
        data['columns'] = '[{}]'.format(','.join(columns))

# either get or post works
#    r = requests.post(url, data=data)
    r = requests.get(url, params=data)

    if verbose:
        print(r.url)
    r.raise_for_status()
    if format == "json":
        return r.json()
    else:
        return r.text


def checklegal(table,release):
    """Checks if this combination of table and release is acceptable
    
    Raises a VelueError exception if there is problem
    """
    
    releaselist = ("dr1", "dr2")
    if release not in ("dr1","dr2"):
        raise ValueError("Bad value for release (must be one of {})".format(', '.join(releaselist)))
    if release=="dr1":
        tablelist = ("mean", "stack")
    else:
        tablelist = ("mean", "stack", "detection")
    if table not in tablelist:
        raise ValueError("Bad value for table (for {} must be one of {})".format(release, ", ".join(tablelist)))


def ps1metadata(table="mean",release="dr1",
           baseurl="https://catalogs.mast.stsci.edu/api/v0.1/panstarrs"):
    """Return metadata for the specified catalog and table
    
    Parameters
    ----------
    table (string): mean, stack, or detection
    release (string): dr1 or dr2
    baseurl: base URL for the request
    
    Returns an astropy table with columns name, type, description
    """
    
    checklegal(table,release)
    url = "{baseurl}/{release}/{table}/metadata".format(**locals())
    r = requests.get(url)
    r.raise_for_status()
    v = r.json()
    # convert to astropy table
    tab = Table(rows=[(x['name'],x['type'],x['description']) for x in v],
               names=('name','type','description'))
    return tab


def mastQuery(request):
    """Perform a MAST query.

    Parameters
    ----------
    request (dictionary): The MAST request json object

    Returns head,content where head is the response HTTP headers, and content is the returned data
    """
    
    server='mast.stsci.edu'

    # Grab Python Version 
    version = ".".join(map(str, sys.version_info[:3]))

    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}

    # Encoding the request as a json string
    requestString = json.dumps(request)
    requestString = urlencode(requestString)
    
    # opening the https connection
    conn = httplib.HTTPSConnection(server)

    # Making the query
    conn.request("POST", "/api/v0/invoke", "request="+requestString, headers)

    # Getting the response
    resp = conn.getresponse()
    head = resp.getheaders()
    content = resp.read().decode('utf-8')

    # Close the https connection
    conn.close()

    return head,content


def resolve(name):
    """Get the RA and Dec for an object using the MAST name resolver
    
    Parameters
    ----------
    name (str): Name of object

    Returns RA, Dec tuple with position"""

    resolverRequest = {'service':'Mast.Name.Lookup',
                       'params':{'input':name,
                                 'format':'json'
                                },
                      }
    headers,resolvedObjectString = mastQuery(resolverRequest)
    resolvedObject = json.loads(resolvedObjectString)
    # The resolver returns a variety of information about the resolved object, 
    # however for our purposes all we need are the RA and Dec
    try:
        objRa = resolvedObject['resolvedCoordinate'][0]['ra']
        objDec = resolvedObject['resolvedCoordinate'][0]['decl']
    except IndexError as e:
        raise ValueError("Unknown object '{}'".format(name))
    return (objRa, objDec)

In [8]:
meta = ps1metadata("mean","dr2")
meta

name,type,description
str17,str12,str142
objName,char,IAU name for this object.
objAltName1,char,Alternate name for this object.
objAltName2,char,Altername name for this object.
objAltName3,char,Altername name for this object.
objID,long,Unique object identifier.
uniquePspsOBid,long,Unique internal PSPS object identifier.
ippObjID,long,IPP internal object identifier.
surveyID,unsignedByte,Survey identifier. Details in the Survey table.
htmID,long,Hierarchical triangular mesh (Szalay 2007) index.


Search an area of sky by right ascention and declination

In [9]:
ra = 187.706
dec = 12.391
radius = 50.0/3600.0
constraints = {'nDetections.gt':1}

# strip blanks and weed out blank and commented-out values
columns = """objID,raMean,decMean,nDetections,ng,nr,ni,nz,ny,
    gMeanPSFMag,rMeanPSFMag,iMeanPSFMag,zMeanPSFMag,yMeanPSFMag""".split(',')
columns = [x.strip() for x in columns]
columns = [x for x in columns if x and not x.startswith('#')]
results = ps1cone(ra,dec,radius,release='dr2',columns=columns,verbose=True,**constraints)
# print first few lines
lines = results.split('\n')
print(len(lines),"rows in results -- first 5 rows:")
print('\n'.join(lines[:6]))

https://catalogs.mast.stsci.edu/api/v0.1/panstarrs/dr2/mean.csv?nDetections.gt=1&ra=187.706&dec=12.391&radius=0.013888888888888888&columns=%5BobjID%2CraMean%2CdecMean%2CnDetections%2Cng%2Cnr%2Cni%2Cnz%2Cny%2CgMeanPSFMag%2CrMeanPSFMag%2CiMeanPSFMag%2CzMeanPSFMag%2CyMeanPSFMag%5D
47 rows in results -- first 5 rows:
objID,raMean,decMean,nDetections,ng,nr,ni,nz,ny,gMeanPSFMag,rMeanPSFMag,iMeanPSFMag,zMeanPSFMag,yMeanPSFMag
122851876947049923,187.69476386968975,12.382817851331282,8,2,1,2,3,0,19.867399215698242,18.59269905090332,19.06920051574707,18.139400482177734,-999.0
122851877008084336,187.70079849589698,12.378285588066493,15,3,2,5,5,0,21.162900924682617,-999.0,18.75429916381836,18.438899993896484,-999.0
122851877018527082,187.70183256713327,12.380539342373664,5,2,2,1,0,0,20.314599990844727,-999.0,18.168699264526367,-999.0,-999.0
122851877049294726,187.70494206905568,12.378523957309678,7,2,2,2,1,0,19.03059959411621,18.453699111938477,18.576099395751953,-999.0,-999.0
122851877062673559,1

Convert to astropy

In [10]:
tab = ascii.read(results)
# improve the format
for filter in 'grizy':
    col = filter+'MeanPSFMag'
    try:
        tab[col].format = ".4f"
        tab[col][tab[col] == -999.0] = np.nan
    except KeyError:
        print("{} not found".format(col))
tab

objID,raMean,decMean,nDetections,ng,nr,ni,nz,ny,gMeanPSFMag,rMeanPSFMag,iMeanPSFMag,zMeanPSFMag,yMeanPSFMag
int64,float64,float64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64
122851876947049923,187.69476386968975,12.382817851331282,8,2,1,2,3,0,19.8674,18.5927,19.0692,18.1394,nan
122851877008084336,187.70079849589698,12.378285588066493,15,3,2,5,5,0,21.1629,nan,18.7543,18.4389,nan
122851877018527082,187.70183256713327,12.380539342373664,5,2,2,1,0,0,20.3146,nan,18.1687,nan,nan
122851877049294726,187.70494206905568,12.378523957309678,7,2,2,2,1,0,19.0306,18.4537,18.5761,nan,nan
122851877062673559,187.70625306033213,12.377545233303906,7,1,2,3,0,1,20.5210,18.9063,18.9627,nan,18.1421
122851877090477761,187.70901150138516,12.381089288872962,29,6,7,11,5,0,19.7468,17.7461,17.4915,17.6221,nan
122851877096718451,187.70969564,12.38158628,2,1,0,1,0,0,20.5734,nan,nan,nan,nan
122851877109888891,187.71098809,12.38200012,2,0,0,2,0,0,nan,nan,nan,nan,nan
122851877115528846,187.71134341,12.38181909,2,1,1,0,0,0,23.3524,17.3302,nan,nan,nan


Query single object by ojject id

In [11]:
results1 = ps1search(release='dr2',columns=columns,verbose=True,objid=122851876947049923)
tab1 = ascii.read(results1)
# improve the format
for filter in 'grizy':
    col = filter+'MeanPSFMag'
    try:
        tab1[col].format = ".4f"
        tab1[col][tab1[col] == -999.0] = np.nan
    except KeyError:
        print("{} not found".format(col))
tab1

https://catalogs.mast.stsci.edu/api/v0.1/panstarrs/dr2/mean.csv?objid=122851876947049923&columns=%5BobjID%2CraMean%2CdecMean%2CnDetections%2Cng%2Cnr%2Cni%2Cnz%2Cny%2CgMeanPSFMag%2CrMeanPSFMag%2CiMeanPSFMag%2CzMeanPSFMag%2CyMeanPSFMag%5D


objID,raMean,decMean,nDetections,ng,nr,ni,nz,ny,gMeanPSFMag,rMeanPSFMag,iMeanPSFMag,zMeanPSFMag,yMeanPSFMag
int64,float64,float64,int64,int64,int64,int64,int64,int64,float64,float64,float64,float64,float64
122851876947049923,187.69476386968975,12.382817851331282,8,2,1,2,3,0,19.8674,18.5927,19.0692,18.1394,nan


Search stack objects at same position